In [1]:
import tkinter as tk
import tkinter.filedialog as filedialog
import whisper
import threading
from tkinterdnd2 import DND_FILES, TkinterDnD
from tkinter import ttk


def drop_inside_list_box(event):
    files = listb.tk.splitlist(event.data)
    for filename in files:
        event.widget.insert('end', filename)

def select_file():
    file_path = filedialog.askopenfilename()
    listb.insert(tk.END, file_path)

def clear_file():
    selected_index = listb.curselection()

    listb.delete(selected_index)    

def clear_all_file():
    listb.delete(0, tk.END)

window = TkinterDnD.Tk()

window.geometry('1000x800')
window.config(bg='lightblue')
window.wm_title("Transcriber")

label = tk.Label(window, text="Welcome to Transcriber\n\n1) Please add audio files to the box below\n2) Select the file you want to transcribe and hit 'transcribe'\n3)The text will appear and will be automatically added to the clipboard\n", justify=tk.LEFT)
label.config(bg='lightblue')
label.pack(side=tk.TOP)

# Drag and drop box
listb = tk.Listbox(window, selectmode=tk.SINGLE, width=100, height=10)
listb.pack()
listb.drop_target_register(DND_FILES)
listb.dnd_bind("<<Drop>>", drop_inside_list_box)


import threading

def transcribe_file():
    if listb.curselection():
        progress_window = tk.Toplevel(window)
        progress_window.title("Transcription in progress")
        screen_width = progress_window.winfo_screenwidth()
        screen_height = progress_window.winfo_screenheight()

        x_pos = (screen_width / 2) - (200 / 2)
        y_pos = (screen_height / 2) - (50 / 2)

        progress_window.geometry("400x50+%d+%d" % (x_pos, y_pos))
        progressbar = ttk.Progressbar(progress_window, orient="horizontal", length=400, mode="indeterminate")
        progressbar.pack()
        progressbar.start()
        progress_window.update()

        def transcribe():

            selected_item = listb.get(listb.curselection())
  
            model = whisper.load_model("tiny.en")

            result = model.transcribe(selected_item)

            text.delete('1.0', tk.END)

            text.insert(tk.END, result["text"])

            text.clipboard_clear()
            text.clipboard_append(text.get('1.0', tk.END))

            progressbar.stop()

            progress_window.destroy()

        thread = threading.Thread(target=transcribe)

        thread.start()

    else:
        text.delete(1.0, tk.END)
        text.insert(tk.END, "Select an item")

button = tk.Button(text='Add File', command=select_file)
button.pack()

button = tk.Button(text='Clear', command=clear_file)
button.pack()

button = tk.Button(text='Clear All', command=clear_all_file)
button.pack()

button = tk.Button(text='Transcribe', command=transcribe_file)
button.pack()

text = tk.Text(wrap=tk.WORD, width=100, height=30)
text.pack()

window.mainloop()




/Users/jamesroberts/Library/Python/3.8/lib/python/site-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
